# Text Classification with DSPy

This notebook demonstrates various text classification approaches using DSPy:
- Sentiment analysis and emotion detection
- Multi-class and multi-label classification
- Few-shot learning for new categories
- Hierarchical classification systems
- Classification with confidence scoring

Text classification is fundamental for content analysis, automated moderation, and understanding user intent.

## Setup and Imports

In [ ]:
import os
import sys
sys.path.append('../../')

import dspy
import json
from typing import List, Dict, Any
from utils import setup_default_lm, print_step, print_result, print_error
from utils.datasets import get_sample_classification_data
from dotenv import load_dotenv

# Load environment variables
load_dotenv('../../.env')

## Configure Language Model

In [ ]:
print_step("Configuring Language Model", "Setting up DSPy with OpenAI")

try:
    lm = setup_default_lm(provider="openai", model="gpt-4o", max_tokens=800)
    dspy.configure(lm=lm)
    print_result("Language model configured successfully!")
except Exception as e:
    print_error(f"Failed to configure language model: {e}")
    print("Make sure you have set your OPENAI_API_KEY in the .env file")

## Basic Sentiment Classification

Let's start with simple sentiment analysis.

In [ ]:
print_step("Basic Sentiment Classification", "Simple positive/negative/neutral classification")

class SentimentClassification(dspy.Signature):
    """Classify the sentiment of text as positive, negative, or neutral."""
    text = dspy.InputField(desc="The text to classify")
    sentiment = dspy.OutputField(desc="Sentiment: positive, negative, or neutral")
    confidence = dspy.OutputField(desc="Confidence level: high, medium, or low")

# Create sentiment classifier
sentiment_classifier = dspy.Predict(SentimentClassification)

# Test with various examples
test_texts = [
    "I absolutely love this product! It's amazing and works perfectly.",
    "This is the worst purchase I've ever made. Complete waste of money.",
    "The product is okay. Nothing special but it does what it's supposed to do.",
    "Incredible customer service and fast delivery. Highly recommended!",
    "Disappointed with the quality. Expected much better for the price."
]

print("Sentiment Classification Results:")
print("=" * 70)

for i, text in enumerate(test_texts, 1):
    result = sentiment_classifier(text=text)
    print(f"\n{i}. Text: {text}")
    print(f"   Sentiment: {result.sentiment} (Confidence: {result.confidence})")

print("\n" + "=" * 70)

## Emotion Detection

Let's create a more granular emotion classification system.

In [ ]:
print_step("Emotion Detection", "Classifying specific emotions")

class EmotionClassification(dspy.Signature):
    """Classify the primary emotion expressed in the text."""
    text = dspy.InputField(desc="The text to analyze")
    primary_emotion = dspy.OutputField(desc="Primary emotion: joy, anger, fear, sadness, surprise, disgust, or neutral")
    secondary_emotions = dspy.OutputField(desc="Any secondary emotions present")
    intensity = dspy.OutputField(desc="Emotional intensity: low, medium, or high")
    reasoning = dspy.OutputField(desc="Brief explanation of the classification")

# Create emotion classifier with reasoning
emotion_classifier = dspy.ChainOfThought(EmotionClassification)

# Test emotional texts
emotional_texts = [
    "I can't believe I won the lottery! This is the best day of my life!",
    "I'm so frustrated with this terrible customer service. They completely ignored my complaints!",
    "I'm really worried about the upcoming presentation. What if I mess up?",
    "I miss my grandmother so much. She passed away last year and I still think about her every day.",
    "Wow, I never expected that plot twist! The movie completely caught me off guard.",
    "This food tastes absolutely disgusting. I can't even finish it."
]

print("Emotion Detection Results:")
print("=" * 80)

for i, text in enumerate(emotional_texts, 1):
    result = emotion_classifier(text=text)
    print(f"\n{i}. Text: {text}")
    print(f"   Primary Emotion: {result.primary_emotion} (Intensity: {result.intensity})")
    print(f"   Secondary Emotions: {result.secondary_emotions}")
    print(f"   Reasoning: {result.reasoning}")
    print("-" * 60)

print("\n" + "=" * 80)

## Multi-Class Topic Classification

Let's classify text into different topic categories.

In [ ]:
print_step("Multi-Class Topic Classification", "Categorizing text by topic")

class TopicClassification(dspy.Signature):
    """Classify text into topic categories."""
    text = dspy.InputField(desc="The text to classify")
    primary_topic = dspy.OutputField(desc="Primary topic: technology, sports, politics, health, entertainment, finance, or other")
    subtopic = dspy.OutputField(desc="More specific subtopic if applicable")
    confidence = dspy.OutputField(desc="Confidence score from 1-10")
    keywords = dspy.OutputField(desc="Key words/phrases that indicate the topic")

# Create topic classifier
topic_classifier = dspy.ChainOfThought(TopicClassification)

# Test with various topic texts
topic_texts = [
    "Apple announced their new iPhone with advanced AI capabilities and improved camera technology.",
    "The Lakers defeated the Warriors 108-102 in last night's basketball game at Staples Center.",
    "The Senate passed a new healthcare bill aimed at reducing prescription drug costs for seniors.",
    "Recent studies show that regular exercise can significantly reduce the risk of heart disease.",
    "The new Marvel movie broke box office records, earning $200 million in its opening weekend.",
    "Stock markets rallied today as inflation data came in lower than expected, with the S&P 500 gaining 2%.",
    "Scientists discovered a new species of deep-sea fish in the Mariana Trench using advanced submersibles."
]

print("Topic Classification Results:")
print("=" * 80)

for i, text in enumerate(topic_texts, 1):
    result = topic_classifier(text=text)
    print(f"\n{i}. Text: {text[:100]}...")
    print(f"   Primary Topic: {result.primary_topic}")
    print(f"   Subtopic: {result.subtopic}")
    print(f"   Confidence: {result.confidence}/10")
    print(f"   Keywords: {result.keywords}")
    print(f"   Reasoning: {result.reasoning}")
    print("-" * 60)

print("\n" + "=" * 80)

## Multi-Label Classification

Let's classify text that may belong to multiple categories simultaneously.

In [ ]:
print_step("Multi-Label Classification", "Assigning multiple labels to text")

class MultiLabelClassification(dspy.Signature):
    """Assign multiple relevant labels to text content."""
    text = dspy.InputField(desc="The text to classify")
    labels = dspy.OutputField(desc="All applicable labels from: business, technology, health, education, environment, politics, sports, entertainment")
    primary_label = dspy.OutputField(desc="The most relevant label")
    label_scores = dspy.OutputField(desc="Relevance score (1-10) for each assigned label")
    justification = dspy.OutputField(desc="Explanation for why each label was assigned")

# Create multi-label classifier
multilabel_classifier = dspy.ChainOfThought(MultiLabelClassification)

# Test with complex multi-topic texts
complex_texts = [
    "A new educational technology startup received $50 million in funding to develop AI-powered learning platforms for schools, potentially revolutionizing how students learn math and science.",
    "The government announced new environmental regulations for tech companies, requiring them to reduce carbon emissions by 30% while maintaining economic growth and job creation.",
    "A professional athlete launched a health and wellness app that uses machine learning to create personalized fitness plans, combining sports expertise with cutting-edge technology.",
    "The entertainment industry is adopting virtual reality technology for immersive movie experiences, while also addressing concerns about screen time and mental health impacts."
]

print("Multi-Label Classification Results:")
print("=" * 80)

for i, text in enumerate(complex_texts, 1):
    result = multilabel_classifier(text=text)
    print(f"\n{i}. Text: {text}")
    print(f"   All Labels: {result.labels}")
    print(f"   Primary Label: {result.primary_label}")
    print(f"   Label Scores: {result.label_scores}")
    print(f"   Justification: {result.justification}")
    print(f"   Reasoning: {result.reasoning}")
    print("-" * 70)

print("\n" + "=" * 80)

## Hierarchical Classification

Let's build a hierarchical classification system with main categories and subcategories.

In [ ]:
print_step("Hierarchical Classification", "Multi-level category classification")

class HierarchicalClassification(dspy.Signature):
    """Classify text into hierarchical categories."""
    text = dspy.InputField(desc="The text to classify")
    level1_category = dspy.OutputField(desc="Top-level category: News, Review, Tutorial, Discussion, or Opinion")
    level2_category = dspy.OutputField(desc="Second-level category based on the top-level choice")
    level3_category = dspy.OutputField(desc="Most specific category if applicable")
    classification_path = dspy.OutputField(desc="Full hierarchical path (e.g., 'News > Technology > AI')")

class HierarchicalClassifier(dspy.Module):
    """Multi-step hierarchical classifier."""
    
    def __init__(self):
        super().__init__()
        
        # Level 1: Content Type
        class Level1Classification(dspy.Signature):
            """Classify content type."""
            text = dspy.InputField(desc="Text to classify")
            content_type = dspy.OutputField(desc="Content type: news, review, tutorial, discussion, or opinion")
            reasoning = dspy.OutputField(desc="Why this content type was chosen")
        
        # Level 2: Domain
        class Level2Classification(dspy.Signature):
            """Classify domain/topic."""
            text = dspy.InputField(desc="Text to classify")
            content_type = dspy.InputField(desc="Already determined content type")
            domain = dspy.OutputField(desc="Domain: technology, health, finance, entertainment, sports, politics, science, or other")
            reasoning = dspy.OutputField(desc="Why this domain was chosen")
        
        # Level 3: Specific Topic
        class Level3Classification(dspy.Signature):
            """Classify specific topic within domain."""
            text = dspy.InputField(desc="Text to classify")
            content_type = dspy.InputField(desc="Content type")
            domain = dspy.InputField(desc="Domain")
            specific_topic = dspy.OutputField(desc="Specific topic within the domain")
            reasoning = dspy.OutputField(desc="Why this specific topic was chosen")
        
        self.level1_classifier = dspy.Predict(Level1Classification)
        self.level2_classifier = dspy.Predict(Level2Classification)
        self.level3_classifier = dspy.Predict(Level3Classification)
    
    def forward(self, text):
        # Level 1: Content Type
        level1_result = self.level1_classifier(text=text)
        
        # Level 2: Domain
        level2_result = self.level2_classifier(
            text=text,
            content_type=level1_result.content_type
        )
        
        # Level 3: Specific Topic
        level3_result = self.level3_classifier(
            text=text,
            content_type=level1_result.content_type,
            domain=level2_result.domain
        )
        
        # Create classification path
        classification_path = f"{level1_result.content_type} > {level2_result.domain} > {level3_result.specific_topic}"
        
        return dspy.Prediction(
            text=text,
            level1_category=level1_result.content_type,
            level1_reasoning=level1_result.reasoning,
            level2_category=level2_result.domain,
            level2_reasoning=level2_result.reasoning,
            level3_category=level3_result.specific_topic,
            level3_reasoning=level3_result.reasoning,
            classification_path=classification_path
        )

# Create hierarchical classifier
hierarchical_classifier = HierarchicalClassifier()

# Test with various content types
hierarchical_texts = [
    "Breaking: Apple announces new AI chip that promises 40% faster machine learning performance in next-generation iPhones.",
    "How to set up a home gym on a budget: A complete guide with equipment recommendations and workout routines.",
    "I think the new climate policies are too aggressive and will hurt small businesses more than help the environment.",
    "The new Spider-Man movie is visually stunning but lacks the emotional depth of previous films. Rating: 7/10.",
    "What are your thoughts on remote work? I've been working from home for 2 years and love the flexibility."
]

print("Hierarchical Classification Results:")
print("=" * 80)

for i, text in enumerate(hierarchical_texts, 1):
    result = hierarchical_classifier(text=text)
    print(f"\n{i}. Text: {text}")
    print(f"   Classification Path: {result.classification_path}")
    print(f"   Level 1 ({result.level1_category}): {result.level1_reasoning}")
    print(f"   Level 2 ({result.level2_category}): {result.level2_reasoning}")
    print(f"   Level 3 ({result.level3_category}): {result.level3_reasoning}")
    print("-" * 70)

print("\n" + "=" * 80)

## Few-Shot Learning for New Categories

Let's demonstrate how to quickly adapt to new classification categories with just a few examples.

In [ ]:
print_step("Few-Shot Learning", "Learning new categories from examples")

class FewShotClassification(dspy.Signature):
    """Classify text using provided examples of categories."""
    examples = dspy.InputField(desc="Examples of each category with their labels")
    text_to_classify = dspy.InputField(desc="New text to classify")
    predicted_category = dspy.OutputField(desc="Predicted category based on the examples")
    confidence = dspy.OutputField(desc="Confidence in the prediction (high/medium/low)")
    reasoning = dspy.OutputField(desc="Explanation comparing the text to the examples")

# Create few-shot classifier
few_shot_classifier = dspy.ChainOfThought(FewShotClassification)

# Define new categories with examples
customer_service_examples = """
Category Examples:

COMPLAINT: "This product broke after one week. I want a full refund immediately!"
COMPLAINT: "Terrible customer service. Nobody answered my calls and my issue is still unresolved."

INQUIRY: "What's the warranty period for this product?"
INQUIRY: "Do you ship to international addresses?"

COMPLIMENT: "Amazing product quality and fast shipping. Very satisfied with my purchase!"
COMPLIMENT: "Outstanding customer support. They solved my problem quickly and professionally."

REQUEST: "Can you help me track my order? The tracking number isn't working."
REQUEST: "I need to change my delivery address for order #12345."
"""

# Test texts for classification
test_customer_texts = [
    "The delivery was supposed to arrive yesterday but it's still not here. What's going on?",
    "Love the new design! The user interface is so much better than the old version.",
    "How do I return an item if I'm not satisfied with it?",
    "This is unacceptable! The product description was completely misleading and now I'm stuck with something I don't need.",
    "Could you please extend my subscription for another month? I forgot to renew it on time."
]

print("Few-Shot Classification Results:")
print("=" * 80)

for i, text in enumerate(test_customer_texts, 1):
    result = few_shot_classifier(
        examples=customer_service_examples,
        text_to_classify=text
    )
    print(f"\n{i}. Text: {text}")
    print(f"   Predicted Category: {result.predicted_category}")
    print(f"   Confidence: {result.confidence}")
    print(f"   Reasoning: {result.reasoning}")
    print("-" * 60)

print("\n" + "=" * 80)

## Classification Evaluation

Let's evaluate our classification systems against labeled data.

In [ ]:
print_step("Classification Evaluation", "Measuring accuracy against ground truth")

def evaluate_classifier(classifier, test_examples, field_name="sentiment"):
    """Evaluate a classifier against labeled examples."""
    correct = 0
    total = len(test_examples)
    results = []
    
    print(f"Evaluating on {total} examples...")
    print("=" * 60)
    
    for i, example in enumerate(test_examples, 1):
        # Get prediction
        if hasattr(classifier, 'forward'):
            prediction = classifier(text=example.text)
        else:
            prediction = classifier(text=example.text)
        
        # Extract predicted value
        if hasattr(prediction, field_name):
            predicted = getattr(prediction, field_name).lower().strip()
        else:
            # Fallback to first field if field_name not found
            predicted = str(list(prediction.__dict__.values())[1]).lower().strip()
        
        # Get expected value
        expected = getattr(example, field_name).lower().strip()
        
        # Check if correct
        is_correct = predicted == expected
        if is_correct:
            correct += 1
        
        results.append({
            "text": example.text,
            "expected": expected,
            "predicted": predicted,
            "correct": is_correct
        })
        
        print(f"{i}. {'✓' if is_correct else '✗'} Expected: {expected}, Predicted: {predicted}")
        if not is_correct:
            print(f"   Text: {example.text[:80]}...")
    
    accuracy = correct / total
    print("\n" + "=" * 60)
    print(f"Accuracy: {accuracy:.3f} ({correct}/{total})")
    
    return accuracy, results

# Get test data
test_examples = get_sample_classification_data()

# Evaluate sentiment classifier
print("Evaluating Basic Sentiment Classifier:")
sentiment_accuracy, sentiment_results = evaluate_classifier(sentiment_classifier, test_examples, "sentiment")

# Create confusion matrix
def create_confusion_matrix(results, field_name="sentiment"):
    """Create a simple confusion matrix."""
    labels = list(set([r["expected"] for r in results] + [r["predicted"] for r in results]))
    labels.sort()
    
    matrix = {}
    for true_label in labels:
        matrix[true_label] = {}
        for pred_label in labels:
            matrix[true_label][pred_label] = 0
    
    for result in results:
        true_label = result["expected"]
        pred_label = result["predicted"]
        matrix[true_label][pred_label] += 1
    
    return matrix, labels

# Display confusion matrix
matrix, labels = create_confusion_matrix(sentiment_results)

print("\nConfusion Matrix:")
print("=" * 40)
print(f"{'':>12}", end="")
for label in labels:
    print(f"{label:>10}", end="")
print()

for true_label in labels:
    print(f"{true_label:>12}", end="")
    for pred_label in labels:
        print(f"{matrix[true_label][pred_label]:>10}", end="")
    print()

print("\n" + "=" * 40)

## Advanced Classification Techniques

Let's explore some advanced classification approaches.

In [ ]:
print_step("Advanced Classification Techniques", "Uncertainty quantification and ensemble methods")

class UncertaintyAwareClassification(dspy.Signature):
    """Classify text with uncertainty quantification."""
    text = dspy.InputField(desc="Text to classify")
    classification = dspy.OutputField(desc="Most likely classification")
    confidence_score = dspy.OutputField(desc="Numerical confidence score (0-100)")
    alternative_classifications = dspy.OutputField(desc="Other possible classifications with their probabilities")
    uncertainty_factors = dspy.OutputField(desc="Factors that contribute to uncertainty")
    recommendation = dspy.OutputField(desc="Recommendation: accept, review, or reject classification")

class EnsembleClassification(dspy.Module):
    """Ensemble classifier combining multiple approaches."""
    
    def __init__(self):
        super().__init__()
        
        # Different classification approaches
        self.sentiment_classifier = dspy.Predict(SentimentClassification)
        self.emotion_classifier = dspy.ChainOfThought(EmotionClassification)
        self.uncertainty_classifier = dspy.ChainOfThought(UncertaintyAwareClassification)
        
        # Ensemble decision maker
        class EnsembleDecision(dspy.Signature):
            """Make final classification decision from multiple classifiers."""
            text = dspy.InputField(desc="Original text")
            sentiment_result = dspy.InputField(desc="Sentiment classification result")
            emotion_result = dspy.InputField(desc="Emotion classification result")
            uncertainty_result = dspy.InputField(desc="Uncertainty-aware classification result")
            final_classification = dspy.OutputField(desc="Final ensemble classification")
            confidence = dspy.OutputField(desc="Overall confidence in the decision")
            reasoning = dspy.OutputField(desc="Explanation of how the decision was made")
        
        self.ensemble_decision = dspy.ChainOfThought(EnsembleDecision)
    
    def forward(self, text):
        # Get results from different classifiers
        sentiment_result = self.sentiment_classifier(text=text)
        emotion_result = self.emotion_classifier(text=text)
        uncertainty_result = self.uncertainty_classifier(text=text)
        
        # Make ensemble decision
        decision = self.ensemble_decision(
            text=text,
            sentiment_result=f"{sentiment_result.sentiment} (confidence: {sentiment_result.confidence})",
            emotion_result=f"{emotion_result.primary_emotion} (intensity: {emotion_result.intensity})",
            uncertainty_result=f"{uncertainty_result.classification} (score: {uncertainty_result.confidence_score})"
        )
        
        return dspy.Prediction(
            text=text,
            sentiment_classification=sentiment_result.sentiment,
            emotion_classification=emotion_result.primary_emotion,
            uncertainty_classification=uncertainty_result.classification,
            uncertainty_score=uncertainty_result.confidence_score,
            uncertainty_factors=uncertainty_result.uncertainty_factors,
            final_classification=decision.final_classification,
            ensemble_confidence=decision.confidence,
            ensemble_reasoning=decision.reasoning
        )

# Create advanced classifiers
uncertainty_classifier = dspy.ChainOfThought(UncertaintyAwareClassification)
ensemble_classifier = EnsembleClassification()

# Test with ambiguous/challenging texts
challenging_texts = [
    "The movie was... interesting. I'm not sure how I feel about it.",
    "Well, it could be worse, I suppose. At least it works sometimes.",
    "I'm excited but also nervous about the upcoming changes at work.",
    "This product is fine. Nothing amazing, nothing terrible. Just fine."
]

print("Uncertainty-Aware Classification:")
print("=" * 70)

for i, text in enumerate(challenging_texts, 1):
    result = uncertainty_classifier(text=text)
    print(f"\n{i}. Text: {text}")
    print(f"   Classification: {result.classification}")
    print(f"   Confidence Score: {result.confidence_score}/100")
    print(f"   Alternatives: {result.alternative_classifications}")
    print(f"   Uncertainty Factors: {result.uncertainty_factors}")
    print(f"   Recommendation: {result.recommendation}")
    print("-" * 50)

print("\n" + "=" * 70)

print("\nEnsemble Classification:")
print("=" * 70)

for i, text in enumerate(challenging_texts[:2], 1):  # Test first 2 texts
    result = ensemble_classifier(text=text)
    print(f"\n{i}. Text: {text}")
    print(f"   Sentiment: {result.sentiment_classification}")
    print(f"   Emotion: {result.emotion_classification}")
    print(f"   Uncertainty: {result.uncertainty_classification} (score: {result.uncertainty_score})")
    print(f"   Final Classification: {result.final_classification}")
    print(f"   Ensemble Confidence: {result.ensemble_confidence}")
    print(f"   Reasoning: {result.ensemble_reasoning}")
    print("-" * 60)

print("\n" + "=" * 70)

## Summary

In this notebook, we explored comprehensive text classification techniques using DSPy:

### Core Classification Types:

1. **Basic Sentiment Analysis** - Simple positive/negative/neutral classification
2. **Emotion Detection** - Granular emotion classification with intensity
3. **Multi-Class Topic Classification** - Categorizing content by subject matter
4. **Multi-Label Classification** - Assigning multiple relevant labels
5. **Hierarchical Classification** - Multi-level categorization systems
6. **Few-Shot Learning** - Quick adaptation to new categories

### Advanced Techniques:

- **Chain of Thought Reasoning** - Explicit reasoning for classification decisions
- **Confidence Scoring** - Quantifying classification certainty
- **Uncertainty Quantification** - Identifying ambiguous cases
- **Ensemble Methods** - Combining multiple classifiers
- **Evaluation Metrics** - Accuracy measurement and confusion matrices

### Key Features Demonstrated:

- **Structured Output** - Consistent classification formats
- **Reasoning Transparency** - Understanding why classifications were made
- **Adaptability** - Easy modification for new domains and categories
- **Robustness** - Handling ambiguous and challenging text
- **Evaluation** - Systematic performance measurement

### Practical Applications:

- **Content Moderation** - Automatically classifying user-generated content
- **Customer Service** - Routing support tickets to appropriate departments
- **Market Research** - Analyzing customer feedback and sentiment
- **News Categorization** - Organizing articles by topic and type
- **Social Media Analysis** - Understanding public opinion and trends

### Next Steps:

- **Fine-tuning** - Use DSPy optimizers to improve classification accuracy
- **Domain Specialization** - Create industry-specific classifiers
- **Real-time Classification** - Implement streaming text classification
- **Active Learning** - Improve classifiers with user feedback
- **Integration** - Connect with content management and workflow systems

This comprehensive classification system shows how DSPy enables building sophisticated, interpretable, and adaptable text classification solutions that can handle complex real-world scenarios with transparency and confidence.